Anjana Kamath Miyar | 011526762

## Introduction - Topic features using LDA ##

One of the primary applications of natural language processing is to automatically extract the topics from large volumes of text documents. I have used the "Liar Liar Pants on Fire" dataset and used LDA Topic modeling to extract most discussed topics.

### Import Packages ###

In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

C:\Users\Pradhan\Anaconda2\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### Data Parsing from csv ###

In [2]:
dataset_train = pd.read_csv('train.tsv', delimiter = '\t', quoting = 3, header=None)
dataset_train.columns = ["id", "label", "statement", "subject", "speaker", "speaker_title", "State", "party_affiliation", "barely_true", "false", "half_true", "mostly_true", "pants_on_fire","context"]
#dataset_train.head(5)
print dataset_train['statement'][:5]

0    Says the Annies List political group supports ...
1    When did the decline of coal start? It started...
2    Hillary Clinton agrees with John McCain "by vo...
3    Health care reform legislation is likely to ma...
4    The economic turnaround started at the end of ...
Name: statement, dtype: object


In [3]:
statement = dataset_train[['statement']]
statement['uuid'] = statement.index
document_news = statement
print len(document_news)
print document_news[:5]

C:\Users\Pradhan\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


10269
                                           statement  uuid
0  Says the Annies List political group supports ...     0
1  When did the decline of coal start? It started...     1
2  Hillary Clinton agrees with John McCain "by vo...     2
3  Health care reform legislation is likely to ma...     3
4  The economic turnaround started at the end of ...     4


## Data Preprocessing ##

##### Tokenization : Convert sentences to words #####
##### Converted words to lower case ##### 
##### Removed Stop words ##### 
##### Stemming is performed on the words to reduce the words to their root form ##### 
##### Lemmatization is performed to change words to first person and change the word to present tense ##### 

In [4]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

def lemmatize_stemming(text):
    stemmer = PorterStemmer()
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

doc_sample = document_news[document_news['uuid'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\nTokenized and lemmatized Document: ')
print(preprocess(doc_sample))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Pradhan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
original document: 
['Says', 'state', 'Sen.', 'Morgan', 'Carrolls', 'votes', 'made', 'it', 'easier', 'to', 'sue', 'doctors,', 'small', 'businesses,', 'even', 'nurses.', 'Putting', 'her', 'own', 'shady', 'interests', 'first.']


Tokenized and lemmatized Document: 
[u'say', u'state', u'morgan', u'carrol', u'vote', u'easier', u'doctor', u'small', u'busi', u'nurs', u'put', u'shadi', u'interest']


### Choosing documents from preprocessed text ###

In [5]:
processed_statement = document_news['statement'].map(preprocess)
processed_statement[:10]

0    [say, anni, list, polit, group, support, trime...
1    [declin, coal, start, start, natur, take, star...
2    [hillari, clinton, agre, john, mccain, vote, g...
3    [health, care, reform, legisl, like, mandat, f...
4                    [econom, turnaround, start, term]
5    [chicago, bear, start, quarterback, year, tota...
6               [dunnam, live, district, repres, year]
7    [person, stage, work, activ, year, pass, russ,...
8    [take, million, oregon, lotteri, fund, port, n...
9    [say, primari, oppon, glenn, grothman, leibham...
Name: statement, dtype: object

### Creating bigrams and trigrams model ###

Bigrams are 2 words frequently occuring in the document and trigrams are 3 words frequently occuring in the document

In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(processed_statement, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[processed_statement], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[processed_statement[0]]])

C:\Users\Pradhan\Anaconda2\lib\site-packages\gensim\models\phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


[u'say', u'anni', u'list', u'polit', u'group', u'support', u'trimest', u'abort', u'demand']


### Counting occurance of each words with their count ##

In [7]:
dictionary = gensim.corpora.Dictionary(processed_statement)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

(3268, u'foul')
(6618, u'interchang')
(4067, u'jihad')
(4065, u'suzann')
(6033, u'lord')
(3757, u'digit')
(1110, u'strictest')
(2860, u'broward')
(3802, u'disturb')
(5860, u'cianci')
(2617, u'whitman')


### Create a dictionary and corpus needed for Topic modeling ###

The 2 main inputs for LDA model are the dictionary and corpus. A unique id is generated for each word in a document. The  corpus outputs the mapping of word id and word frequency

In [8]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_statement]
bow_corpus[4310]

[(5, 1),
 (24, 1),
 (117, 1),
 (152, 1),
 (351, 1),
 (533, 1),
 (706, 1),
 (903, 1),
 (1060, 1)]

In [9]:
# Human readable format of corpus (term-frequency)
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 5 ("say") appears 1 time.
Word 24 ("vote") appears 1 time.
Word 117 ("state") appears 1 time.
Word 152 ("busi") appears 1 time.
Word 351 ("nurs") appears 1 time.
Word 533 ("small") appears 1 time.
Word 706 ("doctor") appears 1 time.
Word 903 ("put") appears 1 time.
Word 1060 ("interest") appears 1 time.


### Applying TF-IDF on bag of words corpus to get vectors ###

In [10]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.3399028377044521),
 (1, 0.49960086945501425),
 (2, 0.40963633375026987),
 (3, 0.46333565607020666),
 (4, 0.40830190020991924),
 (5, 0.10872586224577774),
 (6, 0.27175617970851096)]


### Training the LDA(latent dirichlet allocation) model ###

LDA is a topic modeling approach which considers each document as a collection of topics and each topic as a collection of words. The algorithm takes topics as inputs and creates a topic distribution from the documents and and word distribution from the topics.

In [28]:
#Build LDA model
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=1500, id2word=dictionary, passes=2, workers=2)

C:\Users\Pradhan\Anaconda2\lib\site-packages\gensim\models\ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


### View the topics in LDA model ###

Above LDA model is trained with 10 topics where each topic is a collection of words and each word contributes a certain weightage to that topic. For example Topic 0 comprises of following words 'say', 'state', 'billion' and so on and the weight of word 'say' is 0.076 

In [30]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.244*"muslim" + 0.177*"rubio" + 0.170*"say" + 0.158*"marco" + 0.106*"fact" + 0.076*"consid" + 0.021*"intern" + 0.017*"parti" + 0.014*"close" + 0.013*"compar"
Topic: 1 
Words: 0.032*"million" + 0.030*"person" + 0.029*"grant" + 0.029*"healthcar" + 0.029*"promis" + 0.028*"leav" + 0.027*"care" + 0.027*"feder" + 0.027*"thompson" + 0.024*"tommi"
Topic: 2 
Words: 0.717*"join" + 0.144*"state" + 0.034*"unit" + 0.008*"seven" + 0.006*"democrat" + 0.005*"senat" + 0.004*"group" + 0.004*"opportun" + 0.004*"choos" + 0.004*"republican"
Topic: 3 
Words: 0.339*"safe" + 0.229*"list" + 0.127*"gun" + 0.119*"peopl" + 0.086*"vote" + 0.026*"say" + 0.011*"purchas" + 0.010*"republican" + 0.008*"introduc" + 0.008*"famili"
Topic: 4 
Words: 0.004*"marriag" + 0.003*"texa" + 0.003*"year" + 0.003*"suprem" + 0.002*"licens" + 0.002*"coupl" + 0.002*"pension" + 0.002*"court" + 0.002*"strike" + 0.002*"salari"
Topic: 5 
Words: 0.192*"research" + 0.159*"cancer" + 0.142*"defens" + 0.124*"year" + 0.040*"citi

Words: 0.151*"trillion" + 0.124*"debt" + 0.080*"year" + 0.067*"govern" + 0.054*"cost" + 0.051*"vote" + 0.051*"feder" + 0.043*"congress" + 0.036*"ceil" + 0.032*"fund"
Topic: 46 
Words: 0.001*"theyr" + 0.001*"doctor" + 0.001*"ceil" + 0.001*"fulli" + 0.001*"extend" + 0.001*"train" + 0.001*"speed" + 0.001*"despit" + 0.001*"intern" + 0.001*"stadium"
Topic: 47 
Words: 0.001*"theyr" + 0.001*"doctor" + 0.001*"ceil" + 0.001*"fulli" + 0.001*"extend" + 0.001*"train" + 0.001*"speed" + 0.001*"despit" + 0.001*"intern" + 0.001*"stadium"
Topic: 48 
Words: 0.056*"nation" + 0.032*"offic" + 0.030*"econom" + 0.029*"kasich" + 0.028*"growth" + 0.028*"term" + 0.028*"take" + 0.027*"claim" + 0.025*"economi" + 0.023*"john"
Topic: 49 
Words: 0.428*"georg" + 0.306*"bush" + 0.031*"crisi" + 0.031*"go" + 0.031*"offic" + 0.024*"debt" + 0.020*"trillion" + 0.019*"inherit" + 0.015*"million" + 0.014*"walker"
Topic: 50 
Words: 0.324*"coverag" + 0.127*"health" + 0.113*"care" + 0.084*"american" + 0.069*"data" + 0.055*"year"

Words: 0.001*"theyr" + 0.001*"doctor" + 0.001*"ceil" + 0.001*"fulli" + 0.001*"extend" + 0.001*"train" + 0.001*"speed" + 0.001*"despit" + 0.001*"intern" + 0.001*"stadium"
Topic: 91 
Words: 0.208*"health" + 0.088*"care" + 0.085*"individu" + 0.060*"plan" + 0.051*"feder" + 0.049*"purchas" + 0.048*"reform" + 0.047*"board" + 0.047*"benefit" + 0.046*"committe"
Topic: 92 
Words: 0.180*"percent" + 0.150*"bush" + 0.110*"year" + 0.055*"administr" + 0.055*"spend" + 0.055*"georg" + 0.055*"obama" + 0.055*"rais" + 0.054*"budget" + 0.053*"presid"
Topic: 93 
Words: 0.181*"exceed" + 0.153*"packag" + 0.095*"nation" + 0.090*"poverti" + 0.089*"averag" + 0.084*"rate" + 0.059*"percent" + 0.038*"board" + 0.029*"vote" + 0.026*"counti"
Topic: 94 
Words: 0.097*"fight" + 0.095*"pollut" + 0.086*"congressman" + 0.084*"support" + 0.065*"miami" + 0.057*"chang" + 0.050*"climat" + 0.050*"vote" + 0.044*"abil" + 0.042*"say"
Topic: 95 
Words: 0.001*"theyr" + 0.001*"doctor" + 0.001*"ceil" + 0.001*"fulli" + 0.001*"extend" +

Words: 0.001*"theyr" + 0.001*"doctor" + 0.001*"ceil" + 0.001*"fulli" + 0.001*"extend" + 0.001*"train" + 0.001*"speed" + 0.001*"despit" + 0.001*"intern" + 0.001*"stadium"
Topic: 136 
Words: 0.563*"combin" + 0.085*"previou" + 0.075*"america" + 0.055*"presid" + 0.055*"unit" + 0.038*"obama" + 0.035*"state" + 0.034*"term" + 0.029*"current" + 0.008*"rate"
Topic: 137 
Words: 0.524*"receiv" + 0.308*"benefit" + 0.097*"medicar" + 0.020*"dollar" + 0.018*"today" + 0.010*"retir" + 0.009*"coupl" + 0.006*"tax" + 0.000*"premium" + 0.000*"famili"
Topic: 138 
Words: 0.014*"state" + 0.012*"insur" + 0.012*"say" + 0.011*"polici" + 0.011*"allow" + 0.010*"citizen" + 0.009*"privat" + 0.009*"compani" + 0.009*"properti" + 0.008*"vote"
Topic: 139 
Words: 0.435*"requir" + 0.260*"exist" + 0.090*"absolut" + 0.048*"million" + 0.037*"year" + 0.020*"care" + 0.020*"cut" + 0.015*"health" + 0.010*"recess" + 0.008*"peopl"
Topic: 140 
Words: 0.175*"ask" + 0.169*"question" + 0.058*"presid" + 0.052*"peopl" + 0.041*"million" 

Words: 0.004*"care" + 0.003*"health" + 0.003*"medicaid" + 0.003*"women" + 0.002*"cost" + 0.002*"texa" + 0.002*"birth" + 0.002*"plan" + 0.002*"includ" + 0.002*"shoot"
Topic: 181 
Words: 0.278*"earth" + 0.077*"percent" + 0.072*"unemploy" + 0.067*"test" + 0.066*"go" + 0.045*"market" + 0.040*"actual" + 0.038*"nuclear" + 0.038*"rate" + 0.037*"labor"
Topic: 182 
Words: 0.429*"massiv" + 0.061*"complet" + 0.055*"million" + 0.031*"avail" + 0.029*"cost" + 0.029*"process" + 0.026*"year" + 0.023*"staff" + 0.021*"addit" + 0.021*"peopl"
Topic: 183 
Words: 0.275*"bring" + 0.113*"share" + 0.106*"fuel" + 0.076*"histor" + 0.063*"standard" + 0.051*"presid" + 0.050*"obama" + 0.041*"car" + 0.028*"cost" + 0.021*"budget"
Topic: 184 
Words: 0.001*"theyr" + 0.001*"doctor" + 0.001*"ceil" + 0.001*"fulli" + 0.001*"extend" + 0.001*"train" + 0.001*"speed" + 0.001*"despit" + 0.001*"intern" + 0.001*"stadium"
Topic: 185 
Words: 0.372*"date" + 0.058*"campaign" + 0.056*"governor" + 0.051*"budget" + 0.046*"walker" + 0.04

...

Words: 0.371*"unemploy" + 0.151*"consecut" + 0.143*"percent" + 0.066*"month" + 0.051*"rate" + 0.049*"governor" + 0.048*"state" + 0.033*"lowest" + 0.028*"countri" + 0.025*"say"
Topic: 1302 
Words: 0.001*"theyr" + 0.001*"doctor" + 0.001*"ceil" + 0.001*"fulli" + 0.001*"extend" + 0.001*"train" + 0.001*"speed" + 0.001*"despit" + 0.001*"intern" + 0.001*"stadium"
Topic: 1303 
Words: 0.340*"global" + 0.272*"threat" + 0.129*"presid" + 0.061*"go" + 0.031*"terror" + 0.029*"trump" + 0.029*"donald" + 0.019*"face" + 0.018*"list" + 0.018*"economi"
Topic: 1304 
Words: 0.345*"tri" + 0.199*"israel" + 0.163*"iran" + 0.099*"say" + 0.089*"countri" + 0.021*"presid" + 0.020*"saudi" + 0.020*"nuclear" + 0.015*"deal" + 0.006*"program"
Topic: 1305 
Words: 0.173*"discrimin" + 0.121*"perri" + 0.089*"rick" + 0.089*"feder" + 0.078*"say" + 0.067*"write" + 0.065*"peopl" + 0.048*"sign" + 0.048*"actual" + 0.044*"court"
Topic: 1306 
Words: 0.148*"countri" + 0.125*"best" + 0.084*"state" + 0.068*"wisconsin" + 0.056*"year" 

Words: 0.018*"includ" + 0.015*"nation" + 0.014*"econom" + 0.014*"increas" + 0.010*"budget" + 0.010*"revenu" + 0.009*"recoveri" + 0.009*"refuge" + 0.009*"project" + 0.009*"expect"
Topic: 1347 
Words: 0.001*"theyr" + 0.001*"doctor" + 0.001*"ceil" + 0.001*"fulli" + 0.001*"extend" + 0.001*"train" + 0.001*"speed" + 0.001*"despit" + 0.001*"intern" + 0.001*"stadium"
Topic: 1348 
Words: 0.261*"workforc" + 0.205*"work" + 0.105*"labor" + 0.070*"look" + 0.043*"current" + 0.043*"lowest" + 0.032*"level" + 0.031*"rate" + 0.030*"particip" + 0.029*"stand"
Topic: 1349 
Words: 0.039*"congression" + 0.038*"budget" + 0.037*"cut" + 0.035*"health" + 0.034*"deficit" + 0.033*"second" + 0.033*"year" + 0.031*"reform" + 0.031*"legisl" + 0.030*"insur"
Topic: 1350 
Words: 0.019*"rate" + 0.014*"graduat" + 0.012*"nation" + 0.010*"florida" + 0.009*"continu" + 0.009*"high" + 0.008*"increas" + 0.007*"school" + 0.007*"lead" + 0.007*"fastest"
Topic: 1351 
Words: 0.001*"theyr" + 0.001*"doctor" + 0.001*"ceil" + 0.001*"full

Words: 0.678*"union" + 0.130*"state" + 0.092*"address" + 0.021*"time" + 0.021*"presid" + 0.017*"obama" + 0.013*"word" + 0.011*"barack" + 0.001*"sector" + 0.001*"privat"
Topic: 1391 
Words: 0.001*"constitut" + 0.001*"govern" + 0.001*"take" + 0.001*"hillari" + 0.001*"clinton" + 0.001*"illeg" + 0.001*"foreign" + 0.001*"foundat" + 0.001*"secretari" + 0.001*"money"
Topic: 1392 
Words: 0.755*"your" + 0.051*"american" + 0.044*"countri" + 0.020*"give" + 0.020*"like" + 0.019*"unit" + 0.013*"plan" + 0.013*"govern" + 0.011*"central" + 0.007*"say"
Topic: 1393 
Words: 0.104*"studi" + 0.073*"energi" + 0.068*"billion" + 0.066*"say" + 0.054*"percent" + 0.054*"total" + 0.053*"need" + 0.053*"depart" + 0.053*"best" + 0.053*"solar"
Topic: 1394 
Words: 0.001*"theyr" + 0.001*"doctor" + 0.001*"ceil" + 0.001*"fulli" + 0.001*"extend" + 0.001*"train" + 0.001*"speed" + 0.001*"despit" + 0.001*"intern" + 0.001*"stadium"
Topic: 1395 
Words: 0.049*"highest" + 0.041*"nation" + 0.034*"properti" + 0.030*"tax" + 0.022*"

Words: 0.631*"happen" + 0.083*"time" + 0.039*"sector" + 0.036*"come" + 0.032*"governor" + 0.031*"public" + 0.023*"rate" + 0.021*"privat" + 0.020*"creation" + 0.017*"romney"
Topic: 1435 
Words: 0.168*"time" + 0.110*"percent" + 0.087*"spend" + 0.084*"revenu" + 0.059*"govern" + 0.051*"high" + 0.050*"feder" + 0.045*"right" + 0.038*"close" + 0.036*"say"
Topic: 1436 
Words: 0.148*"gallon" + 0.129*"loss" + 0.103*"gener" + 0.103*"take" + 0.102*"studi" + 0.101*"produc" + 0.063*"percent" + 0.045*"research" + 0.028*"mean" + 0.025*"energi"
Topic: 1437 
Words: 0.135*"percent" + 0.126*"stimulu" + 0.090*"govern" + 0.084*"economi" + 0.049*"cost" + 0.048*"guarante" + 0.047*"money" + 0.035*"earli" + 0.032*"health" + 0.027*"care"
Topic: 1438 
Words: 0.001*"theyr" + 0.001*"doctor" + 0.001*"ceil" + 0.001*"fulli" + 0.001*"extend" + 0.001*"train" + 0.001*"speed" + 0.001*"despit" + 0.001*"intern" + 0.001*"stadium"
Topic: 1439 
Words: 0.252*"get" + 0.118*"capita" + 0.111*"percent" + 0.108*"state" + 0.105*"aver

Words: 0.464*"tell" + 0.146*"say" + 0.090*"worker" + 0.027*"feder" + 0.023*"walker" + 0.018*"report" + 0.018*"like" + 0.018*"requir" + 0.017*"healthcar" + 0.016*"wage"
Topic: 1479 
Words: 0.037*"fund" + 0.029*"pelosi" + 0.028*"regul" + 0.028*"vote" + 0.025*"firearm" + 0.024*"liber" + 0.023*"nanci" + 0.022*"republican" + 0.022*"mike" + 0.021*"program"
Topic: 1480 
Words: 0.045*"tell" + 0.044*"say" + 0.043*"inform" + 0.041*"high" + 0.039*"obamacar" + 0.038*"compani" + 0.038*"impact" + 0.038*"think" + 0.038*"insur" + 0.038*"human"
Topic: 1481 
Words: 0.102*"novemb" + 0.091*"obama" + 0.087*"peopl" + 0.076*"need" + 0.071*"say" + 0.068*"american" + 0.066*"elect" + 0.066*"oppon" + 0.063*"hous" + 0.059*"presid"
Topic: 1482 
Words: 0.065*"american" + 0.059*"million" + 0.050*"citi" + 0.041*"obama" + 0.036*"presid" + 0.034*"stamp" + 0.033*"say" + 0.032*"food" + 0.032*"billion" + 0.032*"budget"
Topic: 1483 
Words: 0.001*"mean" + 0.001*"scott" + 0.001*"lose" + 0.001*"billion" + 0.001*"size" + 0.001

### Applying LDA on TF-IDF ###

In [31]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.015*"say" + 0.010*"texa" + 0.010*"clinton" + 0.010*"year" + 0.009*"florida" + 0.009*"hillari" + 0.007*"spend" + 0.007*"state" + 0.007*"health" + 0.006*"govern"
Topic: 1 Word: 0.013*"say" + 0.010*"obama" + 0.010*"state" + 0.010*"romney" + 0.009*"presid" + 0.009*"tax" + 0.008*"mitt" + 0.008*"rais" + 0.007*"campaign" + 0.007*"governor"
Topic: 2 Word: 0.016*"say" + 0.014*"percent" + 0.012*"tax" + 0.010*"state" + 0.010*"vote" + 0.008*"year" + 0.007*"health" + 0.007*"rais" + 0.007*"voter" + 0.007*"spend"
Topic: 3 Word: 0.016*"job" + 0.012*"say" + 0.012*"state" + 0.011*"creat" + 0.011*"wisconsin" + 0.010*"percent" + 0.010*"citi" + 0.009*"scott" + 0.008*"million" + 0.008*"walker"
Topic: 4 Word: 0.017*"percent" + 0.015*"year" + 0.011*"health" + 0.011*"state" + 0.010*"school" + 0.010*"say" + 0.010*"care" + 0.009*"nation" + 0.008*"averag" + 0.008*"incom"
Topic: 5 Word: 0.016*"say" + 0.010*"year" + 0.009*"state" + 0.008*"obama" + 0.008*"unemploy" + 0.007*"presid" + 0.007*"tri" + 0

In [32]:
processed_statement[4310]

[u'say',
 u'state',
 u'morgan',
 u'carrol',
 u'vote',
 u'easier',
 u'doctor',
 u'small',
 u'busi',
 u'nurs',
 u'put',
 u'shadi',
 u'interest']

### Testing using bag of words by applying LDA ###

In [33]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.382366359234	 
Topic: 0.201*"nurs" + 0.170*"home" + 0.122*"say" + 0.053*"go" + 0.053*"vote" + 0.051*"wisconsin" + 0.048*"increas" + 0.044*"taxpay" + 0.037*"state" + 0.023*"budget"

Score: 0.200066655874	 
Topic: 0.426*"busi" + 0.371*"small" + 0.047*"rais" + 0.039*"plan" + 0.025*"tax" + 0.025*"kind" + 0.014*"democrat" + 0.013*"loan" + 0.008*"job" + 0.008*"rat"

Score: 0.116882666945	 
Topic: 0.215*"deni" + 0.157*"put" + 0.123*"care" + 0.117*"medicar" + 0.103*"say" + 0.094*"senior" + 0.056*"obama" + 0.053*"current" + 0.025*"charg" + 0.016*"bureaucrat"

Score: 0.100066676736	 
Topic: 0.440*"week" + 0.239*"special" + 0.106*"interest" + 0.035*"spend" + 0.032*"corpor" + 0.026*"work" + 0.022*"elect" + 0.010*"includ" + 0.009*"centuri" + 0.008*"propos"

Score: 0.0625788122416	 
Topic: 0.269*"releas" + 0.183*"doctor" + 0.085*"benghazi" + 0.078*"press" + 0.074*"republican" + 0.046*"offici" + 0.045*"administr" + 0.042*"tri" + 0.032*"actual" + 0.031*"email"

Score: 0.0384388193488	 
Topic

### Applying LDA on unknown document ###

In [34]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.909978806973	 
Topic: 0.717*"join" + 0.144*"state" + 0.034*"unit" + 0.008*"seven" + 0.006*"democrat" + 0.005*"senat" + 0.004*"group" + 0.004*"opportun" + 0.004*"choos" + 0.004*"republican"

Score: 0.0100043602288	 
Topic: 0.230*"care" + 0.141*"health" + 0.091*"lowest" + 0.088*"percent" + 0.080*"year" + 0.077*"growth" + 0.075*"afford" + 0.040*"spend" + 0.040*"grow" + 0.040*"rate"

Score: 0.0100034987554	 
Topic: 0.192*"research" + 0.159*"cancer" + 0.142*"defens" + 0.124*"year" + 0.040*"citi" + 0.038*"depart" + 0.037*"base" + 0.035*"percent" + 0.033*"offic" + 0.031*"million"

Score: 0.0100028533489	 
Topic: 0.032*"million" + 0.030*"person" + 0.029*"grant" + 0.029*"healthcar" + 0.029*"promis" + 0.028*"leav" + 0.027*"care" + 0.027*"feder" + 0.027*"thompson" + 0.024*"tommi"

Score: 0.0100027713925	 
Topic: 0.277*"attorney" + 0.256*"public" + 0.134*"texa" + 0.094*"prosecut" + 0.060*"offici" + 0.037*"violat" + 0.035*"district" + 0.034*"gener" + 0.025*"judg" + 0.025*"trust"

Score: 0

### Computing model perplexity and coherence score ###

LDA topic model gave a coherence score of 0.21. LDA Mallet Model - Gensim provides a wrapper to implement Mallet’s LDA is said to give better quality of topics

In [35]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(bow_corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_statement, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

('\nPerplexity: ', -14.613025021435064)
('\nCoherence Score: ', 0.4381146190389922)


### Visualization of Topic keywords ###

In [18]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
vis

C:\Users\Pradhan\Anaconda2\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
5      12.077080        1       1 -0.062385 -0.012768
1      11.034909        1       2  0.098324 -0.008678
7      10.657852        1       3 -0.100159 -0.001435
4      10.566960        1       4 -0.063123  0.032018
9      10.455270        1       5  0.008286 -0.006450
3      10.235923        1       6  0.063967  0.072721
0       9.723151        1       7  0.007119 -0.015235
6       9.294259        1       8  0.045265 -0.100928
8       8.041022        1       9 -0.010740 -0.060562
2       7.913579        1      10  0.013445  0.101317, topic_info=     Category         Freq        Term        Total  loglift  logprob
term                                                                 
1001  Default  2700.000000         say  2700.000000  30.0000  30.0000
343   Default   330.000000    democrat   330.000000  29.0000  29.0000
360   Default   606.000000      health   606.000000  28.0000  28.0000
387   Default   508.000000        care   508.000000  27.0000  27.0000
320   Default   369.000000  republican   369.000000  26.0000  26.0000
346   Default  1200.000000        year  1200.000000  25.0000  25.0000
930   Default   282.000000     clinton   282.000000  24.0000  24.0000
450   Default   851.000000       obama   851.000000  23.0000  23.0000
438   Default   199.000000     hillari   199.000000  22.0000  22.0000
964   Default   445.000000     billion   445.000000  21.0000  21.0000
1008  Default   191.000000      romney   191.000000  20.0000  20.0000
894   Default   136.000000        mitt   136.000000  19.0000  19.0000
620   Default  1366.000000       state  1366.000000  18.0000  18.0000
914   Default   117.000000      island   117.000000  17.0000  17.0000
617   Default   107.000000       rhode   107.000000  16.0000  16.0000
792   Default   449.000000       spend   449.000000  15.0000  15.0000
305   Default   293.000000       senat   293.000000  14.0000  14.0000
541   Default   382.000000        texa   382.000000  13.0000  13.0000
417   Default   651.000000        vote   651.000000  12.0000  12.0000
142   Default   311.000000        unit   311.000000  11.0000  11.0000
498   Default   229.000000       insur   229.000000  10.0000  10.0000
770   Default   412.000000     countri   412.000000   9.0000   9.0000
3     Default   238.000000      dollar   238.000000   8.0000   8.0000
636   Default   482.000000         job   482.000000   7.0000   7.0000
167   Default   577.000000       peopl   577.000000   6.0000   6.0000
226   Default   457.000000    american   457.000000   5.0000   5.0000
200   Default   171.000000       trump   171.000000   4.0000   4.0000
119   Default   164.000000      social   164.000000   3.0000   3.0000
89    Default   202.000000       elect   202.000000   2.0000   2.0000
391   Default   262.000000     florida   262.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
230   Topic10    11.817265    recoveri    32.696053   1.5189  -6.1937
391   Topic10    84.048164     florida   262.930603   1.3961  -4.2319
305   Topic10    81.745277       senat   293.311096   1.2590  -4.2597
850   Topic10    56.847046        like   195.104401   1.3034  -4.6229
85    Topic10    62.824379      public   221.968567   1.2744  -4.5229
1007  Topic10    23.946062      revenu    74.261024   1.4048  -5.4875
322   Topic10    22.375418         day    68.816971   1.4131  -5.5553
1010  Topic10    18.317068       drive    56.139229   1.4166  -5.7554
114   Topic10    26.888309    milwauke    96.152908   1.2623  -5.3716
653   Topic10    19.328440     foreign    62.694344   1.3599  -5.7017
33    Topic10    18.624136        rise    59.920624   1.3680  -5.7388
68    Topic10    53.797882        cost   291.785126   0.8458  -4.6780
620   Topic10   142.584625       state  1366.254761   0.2767  -3.7033
1001  Topic10   217.096100         say  2700.417236   0.0158  -3.2829
1026  Topic10   122

### Optimal number of topics for LDA ###

To find the optimal number of topics build LDA models with different value of number of topics k and pick the one with highest coherence

In [36]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        #model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        #model_list.append(model)
        coherencemodel = CoherenceModel(model=lda_model, texts=processed_statement, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [37]:
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=bow_corpus, texts=processed_statement, start=2, limit=40, step=6)

In [38]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [39]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

('Num Topics =', 2, ' has Coherence Value of', 0.4381)
('Num Topics =', 8, ' has Coherence Value of', 0.4381)
('Num Topics =', 14, ' has Coherence Value of', 0.4381)
('Num Topics =', 20, ' has Coherence Value of', 0.4381)
('Num Topics =', 26, ' has Coherence Value of', 0.4381)
('Num Topics =', 32, ' has Coherence Value of', 0.4381)
('Num Topics =', 38, ' has Coherence Value of', 0.4381)


### Finding the dominant topic in each document ###

To find what the topic is about in a given document and to find a topic number that has highest percentage of contribution in that document

In [45]:
def format_topics_sentences(ldamodel=lda_model, corpus=bow_corpus, texts=processed_statement):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=bow_corpus, texts=processed_statement)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

Document_No  Dominant_Topic  Topic_Perc_Contrib  \
0            0           413.0              0.3712   
1            1          1227.0              0.7170   
2            2            49.0              0.1819   
3            3            76.0              0.2037   
4            4          1080.0              0.2502   
5            5          1148.0              0.2338   
6            6           186.0              0.4001   
7            7          1224.0              0.1980   
8            8           121.0              0.2223   
9            9          1421.0              0.2857   

                                            Keywords  \
0  support, abort, hillari, includ, clinton, dema...   
1  start, administr, natur, presid, bush, coal, b...   
2  georg, bush, crisi, go, offic, debt, trillion,...   
3  health, care, governor, say, year, reform, spe...   
4  term, indic, illeg, cross, number, immigr, few...   
5  bear, year, percent, increas, dramat, time, sp...   
6  district, school, say, chines, texa, year, bon...   
7  mandat, say, vote, feingold, russ, super, dont...   
8  oregon, legislatur, say, author, simpli, milli...   
9  primari, say, vote, republican, report, percen...   

                                                Text  
0  [say, anni, list, polit, group, support, trime...  
1  [declin, coal, start, start, natur, take, star...  
2  [hillari, clinton, agre, john, mccain, vote, g...  
3  [health, care, reform, legisl, like, mandat, f...  
4                  [econom, turnaround, start, term]  
5  [chicago, bear, start, quarterback, year, tota...  
6             [dunnam, live, district, repres, year]  
7  [person, stage, work, activ, year, pass, russ,...  
8  [take, million, oregon, lotteri, fund, port, n...  
9  [say, primari, oppon, glenn, grothman, leibham...

In [44]:
print df_dominant_topic.count()

Document_No           10269
Dominant_Topic        10233
Topic_Perc_Contrib    10233
Keywords              10233
Text                  10269
dtype: int64


In [46]:
import pandas
import csv

header = ['Topic_Perc_Contrib', 'Text']
df_dominant_topic.to_csv('output.csv', columns = header )

In [25]:

#Build a Topic model using Gensim’s LDA, calculated weight of each document as a score, printed value in terms of polynomial function and visualized the topics using pyLDAvis. Determined how to choose the optimal model for number number of topics to choose. Finally I aggregated and presented the results to generate insights that are actionable.

In [26]:
### Further Steps ###

#Need to combine LDA and topic modeling score with rest of the factors applied by other team members to calculate final weight of other factors towards overall score

In [27]:
test_news = pd.read_csv('test.tsv', delimiter = '\t', quoting = 3, header=None)
test_news.columns = ["id", "label", "statement", "subject", "speaker", "speaker_title", "State", "party_affiliation", "barely_true", "false", "half_true", "mostly_true", "pants_on_fire","context"]
#dataset_train.head(5)
#print test_news['statement'][:5]

In [28]:


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

countV = CountVectorizer()
train_count = countV.fit_transform(document_news['statement'].values)
y_train = dataset_train['label']

test_count =  countV.transform(test_news['statement'].values)
y_test = test_news['label']

tfidfV = TfidfVectorizer()
train_tfidf = tfidfV.fit_transform(document_news['statement'].values)
test_tfidf =  tfidfV.transform(test_news['statement'].values)


In [29]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import  LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix, f1_score, classification_report
#from sklearn.cross_validation import GridSearchCV
#from sklearn.cross_validation import learning_curve
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

#string to test
doc_new = ['obama is running for president in 2016']

In [30]:
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    See full source and example: 
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

### Naive Bayes Classifier ###


In [31]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix

clf = MultinomialNB() 
clf.fit(train_tfidf, y_train)
pred = clf.predict(test_tfidf)
score = accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)
#cm = confusion_matrix(y_test, pred, labels=['FAKE', 'REAL'])
#plot_confusion_matrix(cm, classes=['FAKE', 'REAL'])


accuracy:   0.242


### Logistic regression classifier ###

In [32]:
log_clf = LogisticRegression() 
log_clf.fit(train_tfidf, y_train)
pred = log_clf.predict(test_tfidf)
score = accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)


accuracy:   0.249


### Linear SVM classfier ###


In [33]:
svm_clf = svm.LinearSVC() 
svm_clf.fit(train_tfidf, y_train)
pred = svm_clf.predict(test_tfidf)
score = accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)

accuracy:   0.245


### Passive Aggressive Classifier ###

In [34]:
from sklearn.linear_model import PassiveAggressiveClassifier

pa_clf = PassiveAggressiveClassifier(n_iter=50)
pa_clf.fit(train_tfidf, y_train)
pred = pa_clf.predict(test_tfidf)
score = accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)

accuracy:   0.229


### Random forest classifier ###

In [35]:
rfc_clf = RandomForestClassifier()
rfc_clf.fit(train_tfidf, y_train)
pred = rfc_clf.predict(test_tfidf)
score = accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)

accuracy:   0.220


In [61]:
#saving best model to the disk
#model_file = 'final_model.sav'
#pickle.dump(log_clf,open(model_file,'wb'))

In [ ]:
import pickle

#doc_new = ['obama is running for president in 2016']
var = input("Please enter the news text you want to verify: ")
print("You entered: " + str(var))
#function to run for prediction
def detecting_fake_news(var):    
#retrieving the best model for prediction call
    load_model = pickle.load(open('final_model.sav', 'rb'))
    prediction = load_model.predict([var])
    prob = load_model.predict_proba([var])
    print("The given statement is ",prediction[0])
    print("The truth probability score is ",prob[0][1])


if __name__ == '__main__':
    detecting_fake_news(var)

### Top 30 vectors in each of the label using tfidf vectorizer and the model with best accuracy logistic regression ###

In [39]:
def most_informative_feature_for_binary_classification(vectorizer, classifier, n=100):
    """
    See: https://stackoverflow.com/a/26980472
    
    Identify most important features if given a vectorizer and binary classifier. Set n to the number
    of weighted features you would like to show. (Note: current implementation merely prints and does not 
    return top classes.)
    """

    class_labels = classifier.classes_
    feature_names = vectorizer.get_feature_names()
    topn_class1 = sorted(zip(classifier.coef_[0], feature_names))[:n]
    topn_class2 = sorted(zip(classifier.coef_[0], feature_names))[-n:]

    for coef, feat in topn_class1:
        print(class_labels[0], coef, feat)

    print()

    for coef, feat in reversed(topn_class2):
        print(class_labels[1], coef, feat)
     
    print()
    
    for coef, feat in topn_class1:
        print(class_labels[2], coef, feat)
    
    print()
    
    for coef, feat in topn_class1:
        print(class_labels[3], coef, feat)
        
    print()
    
    for coef, feat in topn_class1:
        print(class_labels[4], coef, feat)

most_informative_feature_for_binary_classification(tfidfV, log_clf , n=30)

('barely-true', -1.7208295251144572, u'on')
('barely-true', -1.4001156929716647, u'60')
('barely-true', -1.3355361801970862, u'than')
('barely-true', -1.2109690932472672, u'ohio')
('barely-true', -1.1581465964772615, u'black')
('barely-true', -1.1462879740847616, u'only')
('barely-true', -1.1451650247112146, u'rates')
('barely-true', -1.141094242142306, u'close')
('barely-true', -1.1138770176398332, u'just')
('barely-true', -1.0997096796558488, u'record')
('barely-true', -1.091805451217784, u'had')
('barely-true', -1.0907856912985645, u'since')
('barely-true', -1.0275801370272326, u'primary')
('barely-true', -1.0209708345281876, u'more')
('barely-true', -0.9951184636930478, u'never')
('barely-true', -0.9849568059700278, u'every')
('barely-true', -0.981693508297117, u'80')
('barely-true', -0.9800637813164207, u'voters')
('barely-true', -0.9630601765209656, u'right')
('barely-true', -0.9629813682434956, u'increases')
('barely-true', -0.9354046567376026, u'14')
('barely-true', -0.91466430

### Conclusion ###

Model will classify the given test statements into 'True' or 'False' category and also estimates the probability of truth associated with it.